In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader
import time
from sklearn.metrics import accuracy_score
from sympy import simplify_logic
import re

import deep_logic as dl
from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relu_nn import get_reduced_model, prune_features
from deep_logic.utils.psi_nn import prune_equal_fanin
from deep_logic.models.brl import XBRLClassifier
from deep_logic import logic

set_seed(0)

#%%

data = pd.read_csv('data/vdem/V-Dem-CY-Core-v10.csv')

In [2]:
data['country_name_year'] = data['country_name'] + '_' + data['year'].astype(str)
data['country_name_year']

0                   Mexico_1789
1                   Mexico_1790
2                   Mexico_1791
3                   Mexico_1792
4                   Mexico_1793
                  ...          
27008    Piedmont-Sardinia_1857
27009    Piedmont-Sardinia_1858
27010    Piedmont-Sardinia_1859
27011    Piedmont-Sardinia_1860
27012    Piedmont-Sardinia_1861
Name: country_name_year, Length: 27013, dtype: object

In [3]:
data_2000 = data[data['year']>2000].iloc[:, 12:-1].dropna(axis=1)
data_2000

,v2x_polyarchy,v2x_polyarchy_codelow,v2x_polyarchy_codehigh,v2x_polyarchy_sd,v2x_delibdem,v2x_delibdem_codelow,v2x_delibdem_codehigh,v2x_delibdem_sd,v2x_egaldem,v2x_egaldem_codelow,...,v2xcl_slave_sd,v2xel_elecparl,v2xel_elecpres,v2xex_elecleg,v2xlg_leginter,v2xme_altinf,v2xme_altinf_codelow,v2xme_altinf_codehigh,v2xme_altinf_sd,v2x_divparctrl
212,0.698,0.632,0.763,0.067,0.539,0.467,0.626,0.080,0.411,0.349,...,0.556,0,0,1.00,0.0,0.767,0.615,0.855,0.653,1.616
213,0.711,0.667,0.778,0.055,0.543,0.475,0.608,0.064,0.418,0.359,...,0.556,0,0,1.00,0.0,0.808,0.683,0.887,0.649,1.616
214,0.715,0.668,0.762,0.047,0.554,0.497,0.606,0.053,0.418,0.378,...,0.556,1,0,1.00,0.0,0.808,0.683,0.887,0.649,1.616
215,0.720,0.668,0.746,0.039,0.566,0.519,0.605,0.043,0.423,0.386,...,0.556,0,0,1.00,0.0,0.808,0.683,0.887,0.649,1.616
216,0.720,0.668,0.746,0.039,0.585,0.522,0.615,0.047,0.423,0.386,...,0.556,0,0,1.00,0.0,0.808,0.683,0.887,0.649,1.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25618,0.316,0.283,0.348,0.035,0.215,0.173,0.278,0.052,0.255,0.214,...,0.592,1,1,0.85,0.0,0.516,0.381,0.627,0.633,1.602
25619,0.267,0.236,0.286,0.028,0.194,0.152,0.233,0.041,0.234,0.207,...,0.592,1,1,0.85,0.0,0.516,0.381,0.627,0.633,-0.684
25620,0.260,0.236,0.282,0.025,0.168,0.137,0.208,0.034,0.213,0.188,...,0.599,0,0,0.85,0.0,0.437,0.321,0.579,0.628,-0.448
25621,0.262,0.245,0.284,0.020,0.213,0.181,0.264,0.041,0.214,0.192,...,0.608,0,0,0.85,0.0,0.502,0.374,0.620,0.625,-0.586


In [4]:
high_level_indicators = [
    'v2x_polyarchy',
    #'v2x_libdem',
    #'v2x_partipdem',
    'v2x_delibdem',
    'v2x_egaldem'
]
mid_level_indicators = [
    'v2x_api',
    'v2x_mpi',
    'v2x_freexp_altinf',
    'v2x_frassoc_thick',
    'v2x_suffr',
    'v2xel_frefair',
    'v2x_elecoff',
    #'v2x_liberal',
    'v2xcl_rol',
    #'v2x_jucon',
    #'v2xlg_legcon',
    #'v2x_partip',
    'v2x_cspart',
    #'v2xdd_dd',
    #'v2xel_locelec',
    #'v2xel_regelec',
    'v2xdl_delib',
    'v2x_egal',
    'v2xeg_eqprotec',
    'v2xeg_eqaccess',
    'v2xeg_eqdr',
]

drop_list = ['codelow', 'codehigh', 'sd', 'osp', 'nr', 'mean']
low_level_indicators = []
for f in data_2000.columns:
    if f.endswith('_ord') and f not in high_level_indicators and f not in mid_level_indicators:
        low_level_indicators.append(f)


low_level_indicators_continuous = []
for f in data_2000.columns:
    if f.endswith('_codehigh') or f.endswith('_codelow') and f not in high_level_indicators and f not in mid_level_indicators:
        low_level_indicators_continuous.append(f)

print(f'Main {len(high_level_indicators)} - Area {len(mid_level_indicators)} - Raw {len(low_level_indicators)}')

Main 3 - Area 14 - Raw 62


In [5]:
data_low_continuous = data_2000[low_level_indicators_continuous]
data_low_continuous.shape

(3385, 464)

In [6]:
data_low_raw = data_2000[low_level_indicators]
one_hots = []
for indicator in low_level_indicators:
    c = data_low_raw[indicator].values
    n_bins = int(c.max())
    kbin = KBinsDiscretizer(n_bins=n_bins, encode='onehot-dense', strategy='uniform')
    c1h = kbin.fit_transform(c.reshape(-1, 1))
    one_hots.append(c1h)

new_indicator_names = []
for clist, cname in zip(one_hots, low_level_indicators):
    if clist.shape[1] > 1:
        for i in range(clist.shape[1]):
            new_indicator_names.append(f'{cname}_{i}')
    else:
        new_indicator_names.append(f'{cname}')

data_low = pd.DataFrame(np.hstack(one_hots), columns=new_indicator_names)
data_low.describe()

,v2psbars_ord_0,v2psbars_ord_1,v2psbars_ord_2,v2psbars_ord_3,v2psparban_ord_0,v2psparban_ord_1,v2psparban_ord_2,v2psparban_ord_3,v2psorgs_ord_0,v2psorgs_ord_1,...,v2pepwrgen_ord_1,v2pepwrgen_ord_2,v2peedueq_ord_0,v2peedueq_ord_1,v2peedueq_ord_2,v2peedueq_ord_3,v2pehealth_ord_0,v2pehealth_ord_1,v2pehealth_ord_2,v2pehealth_ord_3
count,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,...,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000,3385.000000
mean,0.056425,0.037518,0.080059,0.825997,0.053471,0.029838,0.028951,0.887740,0.044018,0.150960,...,0.230133,0.730871,0.059970,0.312851,0.132939,0.494239,0.038109,0.322304,0.145643,0.493944
std,0.230775,0.190057,0.271425,0.379168,0.225004,0.170164,0.167694,0.315733,0.205165,0.358063,...,0.420980,0.443572,0.237467,0.463723,0.339559,0.500041,0.191488,0.467428,0.352800,0.500037
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
data_mid = data_2000[mid_level_indicators] > 0.5
data_mid.describe()

,v2x_api,v2x_mpi,v2x_freexp_altinf,v2x_frassoc_thick,v2x_suffr,v2xel_frefair,v2x_elecoff,v2xcl_rol,v2x_cspart,v2xdl_delib,v2x_egal,v2xeg_eqprotec,v2xeg_eqaccess,v2xeg_eqdr
count,3385,3385,3385,3385,3385,3385,3385,3385,3385,3385,3385,3385,3385,3385
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,False,True,True,True,True,True,True,True,True,True,True,True,True
freq,2696,2196,2605,2622,3322,1909,2934,2607,2669,2464,2398,2527,2392,2073


In [8]:
data_high = data_2000[high_level_indicators].iloc[:, 0] > 0.5
data_high.describe()

count     3385
unique       2
top       True
freq      1799
Name: v2x_polyarchy, dtype: object

In [9]:
c0c = torch.FloatTensor(data_low_continuous.values)
c0 = torch.FloatTensor(data_low.values)
c1 = torch.FloatTensor(data_mid.values)
c2 = torch.FloatTensor(data_high.values)
print(c0c.shape)
print(c0.shape)
print(c1.shape)
print(c2.shape)

torch.Size([3385, 464])
torch.Size([3385, 241])
torch.Size([3385, 14])
torch.Size([3385])


# Low-to-mid level explanations

In [10]:
results_dir = 'results/vdem_low2mid'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mid_concepts = list(data_mid.columns)
low_concepts = list(data_low.columns)
n_rep = 10
tot_epochs = 6001
prune_epochs = 3001

n_splits = 10
seed = 42
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

x = c0
xh = c1
y = c2

print(low_concepts[:5])
print(mid_concepts[:5])

['v2psbars_ord_0', 'v2psbars_ord_1', 'v2psbars_ord_2', 'v2psbars_ord_3', 'v2psparban_ord_0']
['v2x_api', 'v2x_mpi', 'v2x_freexp_altinf', 'v2x_frassoc_thick', 'v2x_suffr']


## Train loops

In [11]:
def train_nn(x_train, y_train, need_pruning, seed, device, level, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    
    if level == 'mid':
        layers = [
            torch.nn.Linear(x_train.size(1), 50),
            torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
            torch.nn.Linear(50, 30),
            torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
            torch.nn.Linear(30, y_train.size(1)),
            torch.nn.Sigmoid(),
        ]
        loss_form = torch.nn.BCELoss()
    else:
        layers = [
            torch.nn.Linear(x_train.size(1), 20),
            torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
            torch.nn.Linear(10, 1),
            torch.nn.Sigmoid(),
        ]
        loss_form = torch.nn.BCELoss()
            
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
#                 if level == 'mid':
#                     loss += 0.0001 * torch.norm(module.weight, 1)
#                     loss += 0.0001 * torch.norm(module.bias, 1)
                if level == 'high':
                    loss += 0.0001 * torch.norm(module.weight, 1)
                    loss += 0.0001 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning and level == 'high':
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            if level == 'mid':
                y_pred_d = y_pred > 0.5
                accuracy = y_pred_d.eq(y_train).sum().item() / (y_train.size(0) * y_train.size(1))
                print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
            else:
                y_pred_d = y_pred > 0.5
                accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
                print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
#             if level == 'mid':
#                 y_pred_d = y_pred > 0.5
#                 accuracy = y_pred_d.eq(y_train).sum().item() / (y_train.size(0) * y_train.size(1))
#                 print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
#             else:
#                 y_pred_d = torch.argmax(y_pred, dim=1)
#                 accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
#                 print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [12]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, level, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)

    if level == 'mid':
        layers = [
            torch.nn.Linear(x_train.size(1), 50),
            torch.nn.Sigmoid(),
            torch.nn.Linear(50, 30),
            torch.nn.Sigmoid(),
            torch.nn.Linear(30, y_train.size(1)),
            torch.nn.Sigmoid(),
        ]
    else:
        layers = [
            torch.nn.Linear(x_train.size(1), 10),
            torch.nn.Sigmoid(),
            torch.nn.Linear(10, 4),
            torch.nn.Sigmoid(),
            torch.nn.Linear(4, 1),
            torch.nn.Sigmoid(),
        ]
            
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            if level == 'mid':
                y_pred_d = y_pred > 0.5
                accuracy = y_pred_d.eq(y_train).sum().item() / (y_train.size(0) * y_train.size(1))
                print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
            else:
                y_pred_d = y_pred > 0.5
                accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
                print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [13]:
def c_to_y(method, need_pruning, relu, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies_mid = []
    model_accuracies_top = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    explanation_fidelities = []
    explanation_complexities = []
    for split, (train_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.cpu().detach().numpy())):
        print(f'Split [{split+1}/{n_splits}]')
        x_train, x_test = torch.FloatTensor(x[train_index]), torch.FloatTensor(x[test_index])
        xh_train, xh_test = torch.FloatTensor(xh[train_index]), torch.FloatTensor(xh[test_index])
        y_train, y_test = torch.FloatTensor(y[train_index]), torch.FloatTensor(y[test_index])
    
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0
        elapsed_time, elapsed_time_inv = 0, 0
        
        if method == 'tree':
            classifier = DecisionTreeRegressor(random_state=seed)
            classifier.fit(x_train.cpu().detach().numpy(), xh_train.cpu().detach().numpy())
            xh_train_preds = classifier.predict(x_train.cpu().detach().numpy())
            xh_test_preds = classifier.predict(x_test.cpu().detach().numpy())
            
            classifier_h = DecisionTreeClassifier(random_state=seed)
            classifier_h.fit(xh_train_preds, y_train)
            y_preds = classifier_h.predict(xh_test_preds)
            
            model_accuracy_mid = accuracy_score(xh_test.cpu().detach().numpy(), xh_test_preds>0.5)
            model_accuracy_top = accuracy_score(y_test.cpu().detach().numpy(), y_preds)

            target_class = 1
            start = time.time()
            explanation = tree_to_formula(classifier_h, mid_concepts, target_class)
            elapsed_time = time.time() - start
            explanation_accuracy = model_accuracy_top
            explanation_fidelity = accuracy_score(y_test.cpu().detach().numpy(), y_preds)
            explanation_complexity = dl.logic.complexity(explanation)
        
        elif method == 'brl':
            level = 'mid'
            model = train_nn(x_train, xh_train, need_pruning=False, seed=seed, device=device, 
                             level=level, relu=False, verbose=verbose)
            xh_train_preds = model(x_train.to(device)).cpu().detach().numpy()
            xh_test_preds = model(x_test.to(device)).cpu().detach().numpy()
            xh_train_preds = torch.FloatTensor(xh_train_preds)
            xh_test_preds = torch.FloatTensor(xh_test_preds)

            y_train2 = torch.zeros((y_train.shape[0], 2))
            y_train2[:, 0] = 1-y_train
            y_train2[:, 1] = y_train
            train_dataset = TensorDataset(xh_train_preds, y_train2)
            test_dataset = TensorDataset(xh_test_preds, y_test)
            dummy_concepts = [f'feature{i:010}' for i in range(len(mid_concepts))]
            model = XBRLClassifier(name=os.path.join(results_dir, f'{method}_{split}'), 
                                   n_classes=len(y_train.unique()), 
                                   n_features=x_train.shape[1], feature_names=dummy_concepts, 
                                   class_names=['~democrat', 'democrat'], discretize=True)
            results = model.fit(train_dataset, metric=accuracy_score, save=True)
            model_accuracy_mid = 0
            model_accuracy_top = model.evaluate(test_dataset)
            target_class = 1
            global_explanation, elapsed_time = model.get_global_explanation(target_class, concepts=dummy_concepts, return_time=True)
            explanation_fidelity = 0
            explanation_complexity = 0
            if global_explanation:
                explanation = logic.base.replace_names(global_explanation, mid_concepts)
                explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                              target_class, 
                                                                              x=xh_test, y=y_test,
                                                                              metric=accuracy_score)
                explanation_fidelity = dl.logic.fidelity(y_formula, y_test)
                explanation_complexity = dl.logic.complexity(global_explanation)
                
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                level = 'mid'
                model = train_psi_nn(x_train, xh_train, need_pruning, split, device, level, verbose)
                xh_train_preds = model(x_train.to(device)).cpu().detach().numpy()
                xh_test_preds = model(x_test.to(device)).cpu().detach().numpy()
                xh_train_preds = torch.FloatTensor(xh_train_preds)
                xh_test_preds = torch.FloatTensor(xh_test_preds)

                level = 'high'
                model_h = train_psi_nn(xh_train_preds, y_train, need_pruning, split, device, level, verbose)
                y_train_preds = model_h(xh_train_preds.to(device)).cpu().detach().numpy()
                y_test_preds = model_h(xh_test_preds.to(device)).cpu().detach().numpy()

                model_accuracy_mid = accuracy_score(xh_test.cpu().detach().numpy(), xh_test_preds>0.5)
                model_accuracy_top = accuracy_score(y_test.cpu().detach().numpy(), y_test_preds>0.5)

            else:
                level = 'mid'
                model = train_nn(x_train, xh_train, need_pruning, seed, device, level, relu, verbose)
                xh_train_preds = model(x_train.to(device)).cpu().detach().numpy()
                xh_test_preds = model(x_test.to(device)).cpu().detach().numpy()
                xh_train_preds = torch.FloatTensor(xh_train_preds)
                xh_test_preds = torch.FloatTensor(xh_test_preds)

                level = 'high'
                model_h = train_nn(xh_train_preds, y_train, need_pruning, seed, device, level, relu, verbose)
                y_train_preds = model_h(xh_train_preds.to(device)).cpu().detach().numpy()
                y_test_preds = model_h(xh_test_preds.to(device)).cpu().detach().numpy()

                model_accuracy_mid = accuracy_score(xh_test.cpu().detach().numpy(), xh_test_preds>0.5)
                model_accuracy_top = accuracy_score(y_test.cpu().detach().numpy(), y_test_preds>0.5)

            # positive class
            target_class = 1
            start = time.time()
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model_h, device)[0]
            else:
                global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model_h, 
                                                                                    xh_train.to(device), y_train.to(device), 
                                                                                    target_class=target_class,
                                                                                    topk_explanations=2,
                                                                                    method=method, device=device)
            elapsed_time = time.time() - start

            explanation_fidelity = 0
            explanation_complexity = 0
            if global_explanation:
                explanation = logic.base.replace_names(global_explanation, mid_concepts)
                explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                              target_class, 
                                                                              x=xh_test, y=y_test,
                                                                              metric=accuracy_score)
                explanation_fidelity = dl.logic.fidelity(y_formula, y_test)
                explanation_complexity = dl.logic.complexity(global_explanation)

        if verbose:
            print(f'\t Model\'s accuracy (mid): {model_accuracy_mid:.4f} - Model\'s accuracy (top): {model_accuracy_top:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Fidelity: "{explanation_fidelity:.4f}" - Complexity: "{explanation_complexity}"')

        methods.append(method)
        splits.append(split)
        explanations.append(explanation)
        model_accuracies_mid.append(model_accuracy_mid)
        model_accuracies_top.append(model_accuracy_top)
        explanation_accuracies.append(explanation_accuracy)
        explanation_fidelities.append(explanation_fidelity)
        explanation_complexities.append(explanation_complexity)
        elapsed_times.append(elapsed_time)
    
    explanation_consistency = dl.logic.formula_consistency(explanations)
    print(f'Consistency of explanations: {explanation_consistency:.4f}')
    
    
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'model_accuracy_mid': model_accuracies_mid,
        'model_accuracy_top': model_accuracies_top,
        'explanation_accuracy': explanation_accuracies,
        'explanation_fidelity': explanation_fidelities,
        'explanation_complexity': explanation_complexities,
        'explanation_consistency': explanation_consistency,
        'elapsed_time': elapsed_times,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

## General pruning

In [14]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu, verbose=True)
results_pruning

Split [1/10]
	 Epoch 0: train accuracy: 0.5213
	 Epoch 500: train accuracy: 0.9802
	 Epoch 1000: train accuracy: 0.9908
	 Epoch 1500: train accuracy: 0.9946
	 Epoch 2000: train accuracy: 0.9962
	 Epoch 2500: train accuracy: 0.9966
	 Epoch 3000: train accuracy: 0.9968
	 Epoch 3500: train accuracy: 0.9968
	 Epoch 4000: train accuracy: 0.9968
	 Epoch 4500: train accuracy: 0.9968
	 Epoch 5000: train accuracy: 0.9968
	 Epoch 5500: train accuracy: 0.9968
	 Epoch 6000: train accuracy: 0.9968
	 Epoch 0: train accuracy: 0.4685
	 Epoch 500: train accuracy: 0.9471
	 Epoch 1000: train accuracy: 0.9475
	 Epoch 1500: train accuracy: 0.9475
	 Epoch 2000: train accuracy: 0.9475
	 Epoch 2500: train accuracy: 0.9481
	 Epoch 3000: train accuracy: 0.9481
	 Epoch 3500: train accuracy: 0.9471
	 Epoch 4000: train accuracy: 0.9471
	 Epoch 4500: train accuracy: 0.9471
	 Epoch 5000: train accuracy: 0.9471
	 Epoch 5500: train accuracy: 0.9471
	 Epoch 6000: train accuracy: 0.9471
	 Model's accuracy (mid): 0.7552 

	 Epoch 6000: train accuracy: 0.9969
	 Epoch 0: train accuracy: 0.4687
	 Epoch 500: train accuracy: 0.9475
	 Epoch 1000: train accuracy: 0.9465
	 Epoch 1500: train accuracy: 0.9475
	 Epoch 2000: train accuracy: 0.9485
	 Epoch 2500: train accuracy: 0.9498
	 Epoch 3000: train accuracy: 0.9498
	 Epoch 3500: train accuracy: 0.8595
	 Epoch 4000: train accuracy: 0.8595
	 Epoch 4500: train accuracy: 0.8595
	 Epoch 5000: train accuracy: 0.8599
	 Epoch 5500: train accuracy: 0.8602
	 Epoch 6000: train accuracy: 0.8625
	 Model's accuracy (mid): 0.7278 - Model's accuracy (top): 0.8521
	 Class 1 - Global explanation: "(v2x_mpi & v2x_freexp_altinf & v2x_elecoff & v2xcl_rol & v2xeg_eqdr) | (v2x_freexp_altinf & v2x_elecoff & v2xcl_rol & ~v2x_mpi & ~v2xeg_eqdr)" - Accuracy: 0.7988
	 Elapsed time 2.4165401458740234
	 Fidelity: "0.7988" - Complexity: "10"
Split [8/10]
	 Epoch 0: train accuracy: 0.5217
	 Epoch 500: train accuracy: 0.9808
	 Epoch 1000: train accuracy: 0.9917
	 Epoch 1500: train accuracy: 0

,method,split,explanation,model_accuracy_mid,model_accuracy_top,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,pruning,0,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.755162,0.926254,0.917404,0.917404,7,0.625,2.743629
1,pruning,1,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.758112,0.943953,0.870206,0.870206,10,0.625,3.714072
2,pruning,2,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0.749263,0.867257,0.817109,0.817109,12,0.625,2.507271
3,pruning,3,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.743363,0.908555,0.831858,0.831858,8,0.625,2.915249
4,pruning,4,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.746313,0.938053,0.935103,0.935103,7,0.625,2.713786
5,pruning,5,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.715976,0.914201,0.834320,0.834320,10,0.625,3.899576
6,pruning,6,(v2x_mpi & v2x_freexp_altinf & v2x_elecoff & v...,0.727811,0.852071,0.798817,0.798817,10,0.625,2.416540
7,pruning,7,v2x_freexp_altinf & v2x_frassoc_thick & v2xel_...,0.733728,0.923077,0.843195,0.843195,7,0.625,2.666849
8,pruning,8,v2x_freexp_altinf & v2xel_frefair & v2x_elecof...,0.695266,0.911243,0.810651,0.810651,8,0.625,2.974110
9,pruning,9,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0.772189,0.923077,0.852071,0.852071,14,0.625,2.577125


## ReLU

In [15]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method, need_pruning, relu, True)
results_weights

Split [1/10]
	 Epoch 0: train accuracy: 0.5204
	 Epoch 500: train accuracy: 0.9799
	 Epoch 1000: train accuracy: 0.9900
	 Epoch 1500: train accuracy: 0.9936
	 Epoch 2000: train accuracy: 0.9955
	 Epoch 2500: train accuracy: 0.9962
	 Epoch 3000: train accuracy: 0.9966
	 Epoch 3500: train accuracy: 0.9968
	 Epoch 4000: train accuracy: 0.9968
	 Epoch 4500: train accuracy: 0.9968
	 Epoch 5000: train accuracy: 0.9968
	 Epoch 5500: train accuracy: 0.9968
	 Epoch 6000: train accuracy: 0.9968
	 Epoch 0: train accuracy: 0.4685
	 Epoch 500: train accuracy: 0.9471
	 Epoch 1000: train accuracy: 0.9475
	 Epoch 1500: train accuracy: 0.9481
	 Epoch 2000: train accuracy: 0.9481
	 Epoch 2500: train accuracy: 0.9481
	 Epoch 3000: train accuracy: 0.9488
	 Epoch 3500: train accuracy: 0.9494
	 Epoch 4000: train accuracy: 0.9498
	 Epoch 4500: train accuracy: 0.9491
	 Epoch 5000: train accuracy: 0.9498
	 Epoch 5500: train accuracy: 0.9498
	 Epoch 6000: train accuracy: 0.9494
	 Model's accuracy (mid): 0.7493 

	 Epoch 5500: train accuracy: 0.9508
	 Epoch 6000: train accuracy: 0.9508
	 Model's accuracy (mid): 0.7456 - Model's accuracy (top): 0.9201
	 Class 1 - Global explanation: "~v2x_mpi | (v2x_api & v2x_freexp_altinf & v2xel_frefair & v2x_elecoff & v2xcl_rol & v2xeg_eqdr)" - Accuracy: 0.4970
	 Elapsed time 3.2283756732940674
	 Fidelity: "0.4970" - Complexity: "7"
Split [8/10]
	 Epoch 0: train accuracy: 0.5210
	 Epoch 500: train accuracy: 0.9807
	 Epoch 1000: train accuracy: 0.9902
	 Epoch 1500: train accuracy: 0.9944
	 Epoch 2000: train accuracy: 0.9960
	 Epoch 2500: train accuracy: 0.9967
	 Epoch 3000: train accuracy: 0.9970
	 Epoch 3500: train accuracy: 0.9970
	 Epoch 4000: train accuracy: 0.9971
	 Epoch 4500: train accuracy: 0.9971
	 Epoch 5000: train accuracy: 0.9971
	 Epoch 5500: train accuracy: 0.9971
	 Epoch 6000: train accuracy: 0.9970
	 Epoch 0: train accuracy: 0.4687
	 Epoch 500: train accuracy: 0.9455
	 Epoch 1000: train accuracy: 0.9455
	 Epoch 1500: train accuracy: 0.9455
	 Ep

,method,split,explanation,model_accuracy_mid,model_accuracy_top,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,weights,0,~v2x_mpi | (v2x_freexp_altinf & v2x_elecoff & ...,0.749263,0.935103,0.530973,0.530973,4,0.39,2.935150
1,weights,1,~v2x_mpi | (v2x_api & v2x_frassoc_thick & v2xe...,0.749263,0.941003,0.492625,0.492625,6,0.39,3.210461
2,weights,2,~v2x_mpi | (v2xel_frefair & v2xcl_rol),0.746313,0.935103,0.530973,0.530973,3,0.39,2.852406
3,weights,3,v2x_mpi | ~v2x_mpi,0.752212,0.917404,0.530973,0.530973,2,0.39,2.796571
4,weights,4,(v2x_frassoc_thick & ~v2x_mpi) | (v2x_api & v2...,0.746313,0.938053,0.784661,0.784661,6,0.39,4.071111
5,weights,5,~v2x_mpi | (v2x_api & v2xel_frefair & v2xeg_eq...,0.718935,0.899408,0.520710,0.520710,4,0.39,3.098731
6,weights,6,~v2x_mpi | (v2x_api & v2x_freexp_altinf & v2xe...,0.745562,0.920118,0.497041,0.497041,7,0.39,3.228376
7,weights,7,~v2x_mpi | (v2x_api & v2xel_frefair & v2x_elec...,0.757396,0.931953,0.497041,0.497041,5,0.39,3.181524
8,weights,8,~v2x_mpi | (v2xel_frefair & v2xcl_rol),0.721893,0.937870,0.532544,0.532544,3,0.39,2.830465
9,weights,9,v2x_mpi | ~v2x_mpi,0.751479,0.911243,0.529586,0.529586,2,0.39,2.757625


## Psi network

In [16]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=True)
results_psi

Split [1/10]
	 Epoch 0: train accuracy: 0.4649
	 Epoch 500: train accuracy: 0.9104
	 Epoch 1000: train accuracy: 0.9367
	 Epoch 1500: train accuracy: 0.9503
	 Epoch 2000: train accuracy: 0.9622
	 Epoch 2500: train accuracy: 0.9742
	 Epoch 3000: train accuracy: 0.9847
	 Epoch 3500: train accuracy: 0.8533
	 Epoch 4000: train accuracy: 0.8856
	 Epoch 4500: train accuracy: 0.8930
	 Epoch 5000: train accuracy: 0.8943
	 Epoch 5500: train accuracy: 0.8964
	 Epoch 6000: train accuracy: 0.8992
	 Epoch 0: train accuracy: 0.4685
	 Epoch 500: train accuracy: 0.7173
	 Epoch 1000: train accuracy: 0.8546
	 Epoch 1500: train accuracy: 0.8651
	 Epoch 2000: train accuracy: 0.8766
	 Epoch 2500: train accuracy: 0.8775
	 Epoch 3000: train accuracy: 0.8805
	 Epoch 3500: train accuracy: 0.8513
	 Epoch 4000: train accuracy: 0.8480
	 Epoch 4500: train accuracy: 0.8510
	 Epoch 5000: train accuracy: 0.8595
	 Epoch 5500: train accuracy: 0.8618
	 Epoch 6000: train accuracy: 0.8638
	 Model's accuracy (mid): 0.3451 

	 Epoch 500: train accuracy: 0.9026
	 Epoch 1000: train accuracy: 0.9372
	 Epoch 1500: train accuracy: 0.9485
	 Epoch 2000: train accuracy: 0.9651
	 Epoch 2500: train accuracy: 0.9782
	 Epoch 3000: train accuracy: 0.9863
	 Epoch 3500: train accuracy: 0.8006
	 Epoch 4000: train accuracy: 0.8554
	 Epoch 4500: train accuracy: 0.8759
	 Epoch 5000: train accuracy: 0.8800
	 Epoch 5500: train accuracy: 0.8835
	 Epoch 6000: train accuracy: 0.8845
	 Epoch 0: train accuracy: 0.5313
	 Epoch 500: train accuracy: 0.8461
	 Epoch 1000: train accuracy: 0.8661
	 Epoch 1500: train accuracy: 0.8677
	 Epoch 2000: train accuracy: 0.8691
	 Epoch 2500: train accuracy: 0.8710
	 Epoch 3000: train accuracy: 0.8809
	 Epoch 3500: train accuracy: 0.7460
	 Epoch 4000: train accuracy: 0.7699
	 Epoch 4500: train accuracy: 0.7781
	 Epoch 5000: train accuracy: 0.7749
	 Epoch 5500: train accuracy: 0.7870
	 Epoch 6000: train accuracy: 0.7968
	 Model's accuracy (mid): 0.3166 - Model's accuracy (top): 0.7811
	 Class 1 - Gl

,method,split,explanation,model_accuracy_mid,model_accuracy_top,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,psi,0,(v2x_mpi),0.345133,0.867257,0.814159,0.814159,1,0.158333,0.008008
1,psi,1,(v2x_freexp_altinf & v2xcl_rol),0.336283,0.849558,0.828909,0.828909,2,0.158333,0.030918
2,psi,2,(v2xeg_eqaccess & (v2xdl_delib | ~v2x_suffr)),0.353982,0.834808,0.828909,0.828909,3,0.158333,0.047872
3,psi,3,(v2x_elecoff),0.265487,0.764012,0.681416,0.681416,1,0.158333,0.028929
4,psi,4,(v2x_frassoc_thick & v2xeg_eqaccess),0.168142,0.820059,0.876106,0.876106,2,0.158333,0.033015
5,psi,5,(v2x_frassoc_thick & v2xeg_eqaccess),0.378698,0.786982,0.795858,0.795858,2,0.158333,0.027925
6,psi,6,(v2x_egal),0.346154,0.733728,0.710059,0.710059,1,0.158333,0.025931
7,psi,7,(v2xel_frefair),0.316568,0.781065,0.920118,0.920118,1,0.158333,0.021945
8,psi,8,(v2x_freexp_altinf & v2x_cspart & (v2xel_frefa...,0.304734,0.849112,0.881657,0.881657,6,0.158333,0.035935
9,psi,9,(v2xeg_eqdr),0.331361,0.636095,0.647929,0.647929,1,0.158333,0.007012


## Decision tree

In [17]:
method = 'tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu, verbose=False)
results_tree

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]
Consistency of explanations: 0.3686


,method,split,explanation,model_accuracy_mid,model_accuracy_top,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,tree,0,(v2xel_frefair <= 0.33 & v2xcl_rol > 0.90 & v2...,0.749263,0.935103,0.935103,0.935103,305,0.368571,0.000998
1,tree,1,(v2xel_frefair <= 0.33 & v2xdl_delib > 0.58 & ...,0.784661,0.932153,0.932153,0.932153,294,0.368571,0.000997
2,tree,2,(v2xel_frefair <= 0.58 & v2xcl_rol > 0.88 & v2...,0.746313,0.932153,0.932153,0.932153,236,0.368571,0.000998
3,tree,3,(v2xel_frefair <= 0.88 & v2xcl_rol > 0.90 & v2...,0.734513,0.914454,0.914454,0.914454,289,0.368571,0.000998
4,tree,4,(v2xel_frefair <= 0.44 & v2xcl_rol > 0.90 & v2...,0.722714,0.917404,0.917404,0.917404,289,0.368571,0.000000
5,tree,5,(v2xel_frefair <= 0.46 & v2xdl_delib > 0.61 & ...,0.745562,0.905325,0.905325,0.905325,319,0.368571,0.000000
6,tree,6,(v2xel_frefair <= 0.47 & v2xcl_rol > 0.90 & v2...,0.760355,0.899408,0.899408,0.899408,313,0.368571,0.000997
7,tree,7,(v2xel_frefair <= 0.33 & v2xcl_rol > 0.88 & v2...,0.769231,0.926036,0.926036,0.926036,309,0.368571,0.000996
8,tree,8,(v2xel_frefair <= 0.88 & v2xcl_rol > 0.90 & v2...,0.754438,0.928994,0.928994,0.928994,273,0.368571,0.000997
9,tree,9,(v2xel_frefair <= 0.58 & v2xcl_rol > 0.88 & v2...,0.789941,0.920118,0.920118,0.920118,314,0.368571,0.001995


## BRL

In [18]:
method = 'brl'
need_pruning = False
relu = False
results_brl = c_to_y(method, need_pruning, relu, verbose=True)
results_brl

Split [1/10]
	 Epoch 0: train accuracy: 0.5213
	 Epoch 500: train accuracy: 0.9802
	 Epoch 1000: train accuracy: 0.9908
	 Epoch 1500: train accuracy: 0.9946
	 Epoch 2000: train accuracy: 0.9962
	 Epoch 2500: train accuracy: 0.9966
	 Epoch 3000: train accuracy: 0.9968
	 Epoch 3500: train accuracy: 0.9968
	 Epoch 4000: train accuracy: 0.9968
	 Epoch 4500: train accuracy: 0.9968
	 Epoch 5000: train accuracy: 0.9968
	 Epoch 5500: train accuracy: 0.9968
	 Epoch 6000: train accuracy: 0.9968
Labels (3046, 2)
Discretized features
Completed model 1/2!
Completed model 2/2!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy (mid): 0.0000 - Model's accuracy (top): 92.0354
	 Class 1 - Global explanation: "v2x_mpi | (v2x_freexp_altinf & v2x_frassoc_thick & v2xel_frefair & v2x_elecoff & v2xcl_rol)" - Accuracy: 0.9469
	 Elapsed time 0.017922401428222656
	 Fidelity: "0.9469" - Complexity: "6"
Split [2/10]
	 Epoch 0: train accuracy: 0.5215
	 Epoch 500: train accuracy: 0.9804
	 Epoch 1000: train accuracy: 0

	 Epoch 500: train accuracy: 0.9802
	 Epoch 1000: train accuracy: 0.9904
	 Epoch 1500: train accuracy: 0.9948
	 Epoch 2000: train accuracy: 0.9959
	 Epoch 2500: train accuracy: 0.9967
	 Epoch 3000: train accuracy: 0.9968
	 Epoch 3500: train accuracy: 0.9969
	 Epoch 4000: train accuracy: 0.9969
	 Epoch 4500: train accuracy: 0.9969
	 Epoch 5000: train accuracy: 0.9969
	 Epoch 5500: train accuracy: 0.9969
	 Epoch 6000: train accuracy: 0.9969
Labels (3047, 2)
Discretized features
Completed model 1/2!
Completed model 2/2!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy (mid): 0.0000 - Model's accuracy (top): 92.3077
	 Class 1 - Global explanation: "v2x_mpi & v2x_freexp_altinf & v2xcl_rol" - Accuracy: 0.8136
	 Elapsed time 0.10372233390808105
	 Fidelity: "0.8136" - Complexity: "3"
Consistency of explanations: 0.8286


,method,split,explanation,model_accuracy_mid,model_accuracy_top,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,brl,0,v2x_mpi | (v2x_freexp_altinf & v2x_frassoc_thi...,0,92.035398,0.946903,0.946903,6,0.828571,0.017922
1,brl,1,(v2x_mpi & v2x_freexp_altinf & v2xcl_rol) | (v...,0,92.920354,0.955752,0.955752,8,0.828571,0.014960
2,brl,2,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0,91.740413,0.938053,0.938053,10,0.828571,0.018950
3,brl,3,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0,90.855457,0.949853,0.949853,10,0.828571,0.018919
4,brl,4,v2x_mpi | (v2x_freexp_altinf & v2x_frassoc_thi...,0,93.215339,0.961652,0.961652,6,0.828571,0.016954
5,brl,5,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0,90.828402,0.934911,0.934911,9,0.828571,0.014960
6,brl,6,(v2x_mpi & v2x_freexp_altinf & v2xcl_rol) | (v...,0,91.715976,0.943787,0.943787,8,0.828571,0.013962
7,brl,7,(v2x_mpi & v2x_freexp_altinf) | (v2x_freexp_al...,0,92.307692,0.958580,0.958580,8,0.828571,0.032912
8,brl,8,(v2x_mpi & v2x_freexp_altinf & v2x_frassoc_thi...,0,91.420118,0.940828,0.940828,9,0.828571,0.014960
9,brl,9,v2x_mpi & v2x_freexp_altinf & v2xcl_rol,0,92.307692,0.813609,0.813609,3,0.828571,0.103722


# Summary

In [20]:
cols = ['model_accuracy_top', 'explanation_accuracy', 'explanation_fidelity', 
        'explanation_complexity', 'elapsed_time', 'explanation_consistency']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# # lime
# df_mean = results_lime[cols].mean()
# df_sem = results_lime[cols].sem()
# df_mean.columns = mean_cols
# df_sem.columns = sem_cols
# summary_lime = pd.concat([df_mean, df_sem])
# summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

# BRL
df_mean = results_brl[cols].mean()
df_sem = results_brl[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_brl = pd.concat([df_mean, df_sem])
summary_brl.name = 'BRL'

summary = pd.concat([summary_pruning,
                     summary_weights, 
                     summary_psi, 
                     summary_tree,
                     summary_brl], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_top_mean,explanation_accuracy_mean,explanation_fidelity_mean,explanation_complexity_mean,elapsed_time_mean,explanation_consistency_mean,model_accuracy_top_sem,explanation_accuracy_sem,explanation_fidelity_sem,explanation_complexity_sem,elapsed_time_sem,explanation_consistency_sem
pruning,0.910774,0.851073,0.851073,9.3,2.912821,0.625000,0.009276,0.014165,0.014165,0.746101,0.158873,0.000000e+00
weights,0.926726,0.544713,0.544713,4.2,3.096242,0.390000,0.004398,0.027152,0.027152,0.553775,0.122414,0.000000e+00
psi,0.792268,0.798512,0.798512,2.0,0.026749,0.158333,0.021982,0.028681,0.028681,0.494413,0.003884,9.251859e-18
tree,0.921115,0.921115,0.921115,294.1,0.000897,0.368571,0.003808,0.003808,0.003808,7.878875,0.000179,1.850372e-17
BRL,91.934684,0.934393,0.934393,7.7,0.026822,0.828571,0.250310,0.013708,0.013708,0.683943,0.008717,0.000000e+00


In [21]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))